In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime

import sys

sys.path.append('../')

from implementations import *

%load_ext autoreload
%autoreload 2

# **Load the training data**

We load the training data.

In [3]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("../data")

print(f"The data has {x_train.shape[0]} samples and {x_train.shape[1]} features !")

The data has 328135 samples and 321 features !


For each feature, we clean the data so the values make more sense

In [6]:
def replace(arr, old_values, new_values):
    result = arr.copy()

    for old_val, new_val in zip(old_values, new_values):
        result[result == old_val] = new_val

    return result

In [8]:
# Where put district of columbia ? super high PIB

new_rank_state = [27,46,20,34,1,16,23,42,0,4,9,38,40,5,19,30,
                 33,28,24,43,15,11,14,17,12,22,48,35,32,39,8,37,3,
                 12,45,7,29,25,6,44,26,47,18,2,31,50,13,10,41,21,49,51,52]
old_states_name =  [1,2,4,5,6,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,
                    29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56,66,72]

ranking_column = replace(x_train[:,0], old_states_name, new_rank_state)

Next 5 columns are useless, they are about interview date and index of patient
## COLUMNS TO DELETE 2 3 4 5 7 8 9 10 11 12 14 19 20 21 22 23

## POSSIBLE MEAN 13 24 25 26 27 28 29 33 37 59 60 62 63 75 77 78 79 80 81 82 83 84 85 86 89 90 92 93 98 110 111 112 113 114

The 6th column is DISPCODE, so if interview was complete or not, I change value to 1 if complete 0 if not.

In [9]:
cont_data = [13,24,25,26,27,28,29,33,37,59,60,62,63,75,77,78,
             79,80,81,82,83,84,85,86,89,90,92,93,98,110,111,112,113,114,143,147,148,149,150,168,195,
             206,207,208,209,210,211,212,213,248,250,251,253,262,264,266,267,268,269,270,271,276,277,
             285,286,287,288,291,292,293,295,296,297,299,300,301,302,303,304]

## Functions to help conversions of differents answers in one single metric parameter : 

In [12]:
def IntoPounds(x):
    if x >= 9000 :
        return int((x - 9000) * 2.20462)
    else:
        return x 

def IntoInches(x):
    if x < 9000:          
        return np.floor(x/100)*12 + (x % 100)
    else: 
        return (x - 9000) * 0.393701

def WeekToMonth(x):
    x_str = str(x)
    if x_str[0] == "1":       
        return (4.25*int(x_str[-2:])).astype(int)
    else:
        return int(x_str[-2:])
    
def DayToMonth(x):
    x_str = str(x)
    if x_str[0] == "1":       
        return 30 *int(x_str[-2:])
    elif x_str[0] == "2":
        return (4.25*int(x_str[-2:])).astype(int)
    else :
        return int(x_str[-2:])
    
def DayToYear(x):
    x_str = str(x)
    if x_str[0] == "1":       
        return 365 *int(x_str[-2:])
    elif x_str[0] == "2":
        return 52*int(x_str[-2:])
    elif x_str[0] == "3":
        return 12 * int(x_str[-2:])
    else:
        return int(x_str[-2:])

def HourToMinutes(x):
    x_str = str(x)
    return int(x_str[0])*60 + int(x_str[-2:])
    
#def DateType(x):
#    x_str = str(x)
#    year = x_str[-4:]
#    month = x_str[:2] if len(x) == 6 else x_str[0]
#    return np.datetime64(year + '-' + month)

In [13]:
x_train[:,6] = replace(x_train[:,6], [1100,1200], [1,0])
x_train[:,13] = replace(x_train[:,13], [0,1], [1,2])
x_train[:,24] = replace(x_train[:,24], [1,2,7,9], [0,1,np.nan,np.nan])
x_train[:,25] = replace(x_train[:,25], [77,99], [np.nan,np.nan])
x_train[:,26] = replace(x_train[:,26], [2,3,4,5,7,9], [0.75,0.5,0.25,0,np.nan,np.nan])

array_1 = [27,28,29]

for i in array_1 : 
    x_train[:,i] = replace(x_train[:,i], [88,77,99], [np.nan,np.nan,np.nan])

x_train[:,31] = replace(x_train[:,31], [3,7,9], [0,np.nan,np.nan])

array_2 = [30,32,34,35,36,38,39,40,41,42,43,44,45,46,47,48,53,54,55,56,57,61,64,65,66,67,68,69,70,71,72,73,74,87,95,96,100,103,104,107,108,116,117,118]
           
for i in array_2:
    x_train[:,i] = replace(x_train[:,i], [7,9], [np.nan,np.nan])
    

x_train[:,33] = replace(x_train[:,33], [1,2,3,4,7,8,9], [6,18,42,60,np.nan,120,np.nan])
x_train[:,37] = replace(x_train[:,37], [1,2,3,4,7,9], [6,18,42,60,np.nan,np.nan])
x_train[:,49] = replace(x_train[:,49], [98,99], [np.nan,np.nan])

array_3 = [51,52,58]

for i in array_3 : 
    x_train[:,i] = replace(x_train[:,i], [9], [np.nan])
    
x_train[:,59] = replace(x_train[:,59], [88,99], [0,np.nan])
x_train[:,60] = replace(x_train[:,60], [1,2,3,4,5,6,7,8,77,99] , [5,12.5,17.5,22.5,30,42.5,62.5,75,np.nan,np.nan])

array_4 = [62,63]

for i in array_4 :
    x_train[:,i] = replace(x_train[:,i], [7777,9999], [np.nan,np.nan])    
    x_train[:,i] = list(map(IntoPounds,(x_train[:, i])))

x_train[:,75] = replace(x_train[:,75],[1,2,3,4,5,6,7,8,77,99] , [15,60,135,270,1080,2070,3600,np.nan,np.nan,np.nan])
x_train[:,76] = replace(x_train[:,76],[3,7,9] ,[0,np.nan,np.nan])
x_train[:,77] = replace(x_train[:,77],[777,888,999] ,[np.nan,0,np.nan])
x_train[:,77] = list(map(WeekToMonth,(x_train[:, 77])))


array_5 = [78,80,88,91,98,119]

for i in array_5 :
    x_train[:,i] = replace(x_train[:,i], [77,99], [np.nan,np.nan])
    
x_train[:,79] = replace(x_train[:,79],[77,88,99] ,[np.nan,0,np.nan])

array_6 = [81,82,83,84,85,86]

for i in array_6 :
    x_train[:,i] = replace(x_train[:,i], [300,555,777,999],  [0,0,np.nan,np.nan])
    x_train[:,i] = list(map(DayToMonth,(x_train[:, i])))
    
array_7 = [89,90,92,93] 

for i in array_7 :
    x_train[:,i] = replace(x_train[:,i], [777,999],  [0,0,np.nan,np.nan])

x_train[:,89] = list(map(WeekToMonth,(x_train[:, 89])))
x_train[:,90] = list(map(HourToMinutes,(x_train[:, 90])))
x_train[:,92] = list(map(WeekToMonth,(x_train[:, 92])))

array_7 = [94,110,111] 

for i in array_8 :
    x_train[:,i] = replace(x_train[:,i], [777,888,999], [np.nan,0,np.nan])

x_train[:,94] = replace(x_train[:,94], [777,888,999], [np.nan,0,np.nan])
x_train[:,94] = list(map(WeekToMonth,(x_train[:, 94])))
x_train[:,97] = replace(x_train[:,97], [2,3,7,9], [0.5,0,np.nan,np.nan])
x_train[:,99] = replace(x_train[:,99], [2,3,4,5,7,8,9], [0.75,0.5,0.25,0,np.nan,np.nan,np.nan])
x_train[:,101] = replace(x_train[:,101], [777777, 999999],  [np.nan,np.nan])

#x_train[:,101] = list(map(DateType,(x_train[:, 101])))

x_train[:,105] = replace(x_train[:,105], [777777, 999999],  [np.nan,np.nan])
#x_train[:,105] = list(map(DateType,(x_train[:, 105])))

x_train[:,110] = list(map(DayToYear,(x_train[:, 110])))
x_train[:,111] = list(map(DayToYear,(x_train[:, 111])))

x_train[:,113] = replace(x_train[:,113],[77,88,98,99] ,[np.nan,0,np.nan,np.nan])
x_train[:,114] = replace(x_train[:,114],[77,88,99] ,[np.nan,0,np.nan])
x_train[:,115] = replace(x_train[:,114],[1,2,3,4,7,8,9] ,[15,180,540,720,np.nan,0,np.nan])



ValueError: invalid literal for int() with base 10: '.0'

In [14]:
def convert_to_days(x):
    x = np.where((x >= 101) & (x < 200), x-100,x)
    x =  np.where((x >= 201) & (x < 300), x-200,x)
    x =  np.where((x >= 301) & (x < 400), x-300,x)
    x =  np.where((x >= 401) & (x < 500), x-400,x)

    return x

def asthme(x) : 
    if x <= 97:
        return 1
    else :
        return 0 

def line191(x) :
    if x >= 11: 
        return 

#def line252(x): 
#    if (x >= 1 and x <=650) :

In [19]:
nan79 = [120,121,123,124,125,126,129,132,136,137,138,139,
         140,141,142,144,151,154,155,156,157,158,159,160,161,162,163,164,165,166,169,
         170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,
         194,196,198,199,201,202,203,204,205,214,261]
         
for i in nan79 :
    x_train[:,i ] = replace(x_train[:,i ],[7,9] ,[np.nan,np.nan])          
         
nan789 = [192,193]
          
for i in nan789 :
    x_train[:,i ] = replace(x_train[:,i ],[7,8,9] ,[np.nan,np.nan,np.nan])   
         
nan7799 = [122,130,168,224,240]

for i in nan7799 :
    x_train[:,i ] = replace(x_train[:,i ],[77,99] ,[np.nan,np.nan])     
         
x_train[:,127 ] = replace(x_train[:,127 ],[6,7,9] ,[np.nan,np.nan])
x_train[:,128 ] = replace(x_train[:,128 ],[6,7] ,[np.nan,np.nan])

        
nan9 = [131,153,200,223,230,231,232,233,234,235,236,241,242,243,244,255,256,257,258,259,260,263,
        265,278,279,298, 305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320] 

for i in nan9 :
    x_train[:,i ] = replace(x_train[:,i ],[9] ,[np.nan])          
         
nan7 = [133,134,135,146,152]

for i in nan7 :
    x_train[:,i ] = replace(x_train[:,i ],[7] ,[np.nan])    

nan99900 = [264,287,288,293,294,297]
for i in nan99900 :
    x_train[:,i ] = replace(x_train[:,i ],[99900] ,[np.nan]) 

        
x_train[:,143] = replace(x_train[:,143],[555,777,999] ,[20,np.nan,np.nan])
x_train[:,143] = list(map(convert_to_days,(x_train[:, 143])))
        
#DEMANDER AHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH        
x_train[:,145] = list(map(asthme,(x_train[:, 145])))        

n088_98 = [147,148]

for i in n088_98 :     
    x_train[:,i] = replace(x_train[:,i],[88,98] ,[0,np.nan])              

x_train[:,149] = replace(x_train[:,149],[88,98,99] ,[0,np.nan,np.nan])
x_train[:,150] = replace(x_train[:,150],[777,888,999] ,[np.nan,0,np.nan])
           
x_train[:,191] = list(map(line191,(x_train[:, 191])))
           
           
x_train[:,195] = replace(x_train[:,195],[97,98,99] ,[np.nan,0,np.nan])    
x_train[:,197] = replace(x_train[:,197],[97,98,99] ,[np.nan,0,np.nan])   
    
nan088 = [206,207,208,209,210,211,212,213]
for i in nan088 : 
    x_train[:,i] = replace(x_train[:,i],[77,88,99] ,[np.nan,0,np.nan])   

x_train[:,225] = replace(x_train[:,225],[7,77,99] ,[np.nan,np.nan,np.nan])  
x_train[:,239] = replace(x_train[:,239],[7,77,99] ,[np.nan,np.nan,np.nan])   
    
x_train[:,246 ] = replace(x_train[:,246 ],[14] ,[np.nan])        
x_train[:,247 ] = replace(x_train[:,247 ],[3] ,[np.nan])     
x_train[:,262 ] = replace(x_train[:,262 ],[900] ,[np.nan])    

We clean the data by removing the nan values by the mean of the rest of the feature

In [ ]:
data = np.ones(x_train.shape)
stds = np.array([])
for i in range(data.shape[1]):
    d, std = standardize_clean(x_train[:, i])
    data[:, i] = d
    stds = np.append(stds, std)
print(stds.shape)

We further clean the data by removing the features where the variance is zero since they are constants for all samples

In [ ]:
indices = np.where(stds != 0)
data_var = data[:, indices]
data_var = np.squeeze(data_var, axis = 1)
print(data_var.shape)

We also remove the 8 first features as the appear weird in the task of predicting a heart attack

In [ ]:
data_cleaned = data_var[:, 9:]
print(data_cleaned.shape)

We then separe the data to train on 60% of the total and to test it on the remaining 40% 

In [ ]:
train_size = np.floor(data_cleaned.shape[0] * 0.6).astype(int)
data_cross = data_cleaned[:train_size, :]
pred_cross = pred[:train_size]
data_test = data_cleaned[train_size:, :]
pred_test = pred[train_size:]
print(f"Cross shape : {data_cross.shape} ; Test shape : {data_test.shape}")

Now we build our models for linear regression

In [ ]:
y, tx = build_model_data(data_cross, pred_cross)
y_test, tx_test = build_model_data(data_test, pred_test)
print(f"The data has now {tx.shape[1]} features !")